# WTB

Automatic classification of birds for this project : http://www.cabane-oiseaux.org/

## Data cooking
First, prepare the data : we need a train dataset and a validation dataset.
We use the Keras ImageDataGenerator that can build a training set from directories containing images.
Each sub directory must contains a category of bird : Keras will associate each one to a category in the model.

In [58]:
from PIL import Image
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import os
import numpy as np

# Directory that contains images : each subdirectory contains a category of bird
DIR = '/home/kvjw3322/Documents/Prez/ML4Noobs/Notebooks/images'

# Generator for train
train_image_generator = ImageDataGenerator()
train_iterator = train_image_generator.flow_from_directory(
    DIR, # Root directory
    target_size=(150, 150), # Images will be processed to this size
    batch_size=32, # How many images are processed at the same time ?
    class_mode='categorical') # Each subdir is a category

# Generator for validation
valid_image_generator = ImageDataGenerator()
valid_iterator = valid_image_generator.flow_from_directory(DIR,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')


Found 3032 images belonging to 6 classes.
Found 3032 images belonging to 6 classes.


# Prepare human readable predictions

In [68]:
# Map a prediction indice to a label
mapper = { v:k for k,v in train_iterator.class_indices.items()}

def get_human_prediction(model, input_picture_array):
     return mapper.get([ idx[0] for idx, item in np.ndenumerate(model.predict(input_picture_array)[0]) if item == 1.0 ][0])

## RNN : layer by layer
We build the RNN, adding layers one by one

In [59]:
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.models import Model, Sequential

model = Sequential() 

# The input_shape is 150,150 to match the images size. The last dimension is '3' 
# because each pixel is represented by 3 values (RGB). 
# For grayscale images, this parameter would have been '1' (one value to store the grey level)
# We choose to use 32 filters (or feature maps) : all the filters are appyed to the source image at the same time.
# The convolution layer "learn" the values of each filter (as synaptic coefficients are in a Dense hidden layer).
# The kernel_size is the matrix size of the filter
model.add(Conv2D(filters=32, input_shape=(150, 150, 3), kernel_size=(5,5)))

# The subsampling layer get the best value (max with maxpooling) of a set of x pixels. This set comes from a 
# sub matrix of "pool_size"
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))

# Transform the output matrix into a vector for the Dense layer
model.add(Flatten())

model.add(Dense(1024, activation="relu"))
model.add(Dense(6, activation="softmax"))

## Let's learn !
It's time to make our network learn : we compile it (as Keras ask us to), and run the learning from our data.

In [61]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(generator=train_iterator, 
                    validation_data=valid_iterator,
                    validation_steps=10,
                    epochs=10,
                    steps_per_epoch=10,
                    verbose=2)

Epoch 1/10
 - 42s - loss: 7.9074 - acc: 0.5094 - val_loss: 6.7998 - val_acc: 0.5781
Epoch 2/10
 - 35s - loss: 8.2605 - acc: 0.4875 - val_loss: 7.8072 - val_acc: 0.5156
Epoch 3/10
 - 38s - loss: 7.3035 - acc: 0.5469 - val_loss: 7.2028 - val_acc: 0.5531
Epoch 4/10
 - 37s - loss: 7.1524 - acc: 0.5563 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/10
 - 37s - loss: 8.3613 - acc: 0.4813 - val_loss: 7.8576 - val_acc: 0.5125
Epoch 6/10
 - 37s - loss: 7.3035 - acc: 0.5469 - val_loss: 8.2102 - val_acc: 0.4906
Epoch 7/10
 - 36s - loss: 8.5627 - acc: 0.4688 - val_loss: 8.1094 - val_acc: 0.4969
Epoch 8/10
 - 37s - loss: 8.0590 - acc: 0.5000 - val_loss: 7.9079 - val_acc: 0.5094
Epoch 9/10
 - 35s - loss: 7.7568 - acc: 0.5187 - val_loss: 8.5124 - val_acc: 0.4719
Epoch 10/10
 - 36s - loss: 8.3162 - acc: 0.4840 - val_loss: 8.2140 - val_acc: 0.4904


# Model unit validation
Let's get a model prediction from a given picture

In [69]:
picture_path = '/home/kvjw3322/Documents/Prez/WTB/images/mesange bleu/01-20160116_092159-00.jpg'
#picture_path = '/home/kvjw3322/Documents/Prez/WTB/images/rouge gorge/01-20160116_094045-00.jpg'

picture = Image.open(picture_path)
picture = picture.resize(size=(150,150))
picture_array = img_to_array(img=picture)
picture_array = np.expand_dims(picture_array, axis=0)
get_human_prediction(model, picture_array)

'rouge gorge'